In [1]:
import sys
!{sys.executable} -m pip install psycopg2
import psycopg2


def create_tables():
    
    commands = (
        create table inventory(
        IDint,
        cost numeric(8,2),
        leadTime interval,
        category varchar(32),
        Primary key (ID)
        );

        Create table Employee (
        firstName varchar(32),
        lastName varchar(32),
        ssn int,
        employeeID int,
        Salary int,
        salaryType ENUM(“Hourly”, “Salary”),
        userID int,
        priviledge ENUM(‘admin’, ‘sales’, ‘HR’, ‘engineer’),
        loginTime timestamp,
        logoutTime timestamp,
        Primary key (employeeID)
        );

        Create table model(
        mNumber int,
        salePrice float(8,2),
        Primary key (mNumber)
        );


        create table customer (
        customerId int,
        firstName varchar(20),
        lastName varchar(20),
        Primary key (customerID)
        );

        create table order (
        orderNumber	int,
        saleValue	int,
        Primary key (orderNumber),
        employeeID	int,
        customerID	int,
        mNumber int,
        Foreign key (employeeID) references Employee(employeeID),
        Foreign key (customerID) references Customer(customerID),
        Foreign key (mNumber) references Model(mNumber)
        );

        create table composes(
        ID int,
        mNumber	int,
        foreign key (ID) references Inventory(ID),
        foreign key (mNumber) references Model(mNumber),
        Primary key (ID, mNumber)
        );
)
    
    conn = None
    userName = input("Username: ")
    passWord = input("Password: ")
    
    try:
        # connect to PostgreSQL
        conn = psycopg2.connect(host="localhost",database="testdb", user=userName, password=passWord)
        cur = conn.cursor()
        
        # create each table
        for command in commands:
            cur.execute(command)
            
        cur.close()
        conn.commit()
        
    #Handle exceptions    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    #Ensure closed
    finally:
        if conn is not None:
            conn.close()

            
def get_something():
    conn = None
    try:
        conn = psycopg2.connect(host="localhost",database="testdb", user="postgres", password="postgres")
        cur = conn.cursor()
        
        
        cur.execute("SELECT ...")
        
        
        print("The number of rows: ", cur.rowcount)
        row = cur.fetchone()

        while row is not None:
            print(row)
            row = cur.fetchone()

        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            
def insert_X(Y):
"""Insert attribute Y into table X and return row"""
    sql = """INSERT INTO X(Y)
             VALUES(%s) RETURNING Z;"""
    conn = None
    Z = None
    try:
        conn = psycopg2.connect(host="localhost",database="testdb", user="postgres", password="postgres")
        cur = conn.cursor()
        
        
        cur.execute(sql, (Y,))
        
        Z = cur.fetchone()
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

    return Z
            
if __name__ == '__main__':
    create_tables()


IndentationError: expected an indented block (<ipython-input-1-31204914dd9f>, line 64)